In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("Adops & Data Scientist Sample Data - Q1 Analytics.csv")
df.shape

(3553, 4)

1.Site_id with largest number of unique users

In [3]:
data = df[df["country_id"] == "BDV"]
count_user = data.groupby("site_id")["user_id"].nunique()
count_user

site_id
3POLC      2
5NPAU    544
N0OTG     90
Name: user_id, dtype: int64

In [4]:
print("Site_id with the largest number of unique users: ",count_user.idxmax())
print("Associated number: ",str(count_user.max()))

Site_id with the largest number of unique users:  5NPAU
Associated number:  544


2.Four users visiting some site more than 10 times during a certain period

In [5]:
data = df.set_index("ts").loc["2019-02-03 00:00:00":"2019-02-04 23:59:59"]
x = data.groupby(["user_id","site_id"]).count().reset_index().rename(columns={"country_id":"count"})
more_than_10x = x[x["count"]>10]
more_than_10x

,user_id,site_id,count
3,LC06C3,N0OTG,25
417,LC3A59,N0OTG,26
485,LC3C7E,3POLC,15
493,LC3C9D,N0OTG,17


In [6]:
triples = list(more_than_10x.to_records(index=False))
triples

[('LC06C3', 'N0OTG', 25),
 ('LC3A59', 'N0OTG', 26),
 ('LC3C7E', '3POLC', 15),
 ('LC3C9D', 'N0OTG', 17)]

3.Unique number of users' last visit to each site

In [7]:
data = df.sort_values("ts").drop_duplicates(subset=["user_id"], keep="last")
last_visit = data.groupby("site_id")["user_id"].count().reset_index().rename(columns={"user_id":"count"}).sort_values("count",ascending = False)
last_visit

,site_id,count
1,5NPAU,992
5,N0OTG,561
6,QGO3G,289
3,GVOFK,42
0,3POLC,28
7,RT9Z6,2
2,EUZ/Q,1
4,JSUUP,1


In [8]:
top3 = list(last_visit.head(3).to_records(index=False))
top3

[('5NPAU', 992), ('N0OTG', 561), ('QGO3G', 289)]

4.First & Last visits

In [9]:
data = df.sort_values("ts").groupby("user_id",as_index=True)
first = data.first().drop("country_id",axis=1).rename(columns={"site_id":"first_site","ts":"first_ts"})
last = data.last().drop("country_id",axis=1).rename(columns={"site_id":"last_site","ts":"last_ts"})
join = first.join(last)
join.head()

,first_ts,first_site,last_ts,last_site
user_id,,,,
LC00C3,2019-02-03 18:52:50,5NPAU,2019-02-03 18:52:50,5NPAU
LC01C3,2019-02-04 11:35:10,5NPAU,2019-02-04 11:35:10,5NPAU
LC05C3,2019-02-02 14:14:44,5NPAU,2019-02-02 14:14:44,5NPAU
LC06C3,2019-02-01 22:49:39,N0OTG,2019-02-07 01:16:12,N0OTG
LC07C3,2019-02-05 19:06:42,5NPAU,2019-02-05 19:06:42,5NPAU


In [10]:
#here i assume fist & last sites are the same if the user only visit once
count = sum(join["first_site"]==join["last_site"])
print("Number of users having the same first/last visits: ", str(count))

Number of users having the same first/last visits:  1670


In [11]:
#here i only consider users who visit more than once
count = sum((join["first_ts"]!=join["last_ts"]) & (join["first_site"]==join["last_site"]))
print("Number of users having the same first/last visits: ", str(count))

Number of users having the same first/last visits:  409
